## Importing libraries

In [1]:
import pandas as pd
import os
import spacy
from spacy.tokenizer import Tokenizer

import re
import string
import unicodedata


In [2]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [3]:
os.listdir('Assets')


['reutersNLTK.xlsx']

### Reading the first text of the dataframe

In [112]:
text_list = []
df = pd.read_excel('Assets/reutersNLTK.xlsx')
first_text = df.iloc[0,2]


In [113]:
for i in range(len(df)):
    text_list.append(df.iloc[i,2])


In [114]:
len(text_list)

10788

In [115]:
normalized_text = ' '.join(re.sub(' +', ' ',first_text.replace('\'s','').replace('\'t','').replace('&lt;','').replace(">","")).split('\n')[1:]).strip()

spacy. load() is a convenience wrapper that reads the pipeline's config. cfg , uses the language and pipeline information to construct a Language object, loads in the model data and weights, and returns it.

In [106]:
nlp = spacy.load('en_core_web_lg')
tokenizer = nlp.tokenizer

In [107]:
def text_preprocessor(text):
    text = text.lower()
    
    return text
print(text_preprocessor("LOL"))


lol


In [108]:
def  accented_char_removal(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

normalized_text = accented_char_removal(normalized_text)

In [110]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

normalized_text = expand_contractions(normalized_text)
normalized_text


'Mounting trade friction between the  U.S. And Japan has raised fears among many of Asia exporting  nations that the row could inflict far-reaching economic  damage, businessmen and officials said.  They told Reuter correspondents in Asian capitals a U.S.  Move against Japan might boost protectionist sentiment in the  U.S. And lead to curbs on American imports of their products.  But some exporters said that while the conflict would hurt  them in the long-run, in the short-term Tokyo loss might be  their gain.  The U.S. Has said it will impose 300 mln dlrs of tariffs on  imports of Japanese electronics goods on April 17, in  retaliation for Japan alleged failure to stick to a pact not  to sell semiconductors on world markets at below cost.  Unofficial Japanese estimates put the impact of the tariffs  at 10 billion dlrs and spokesmen for major electronics firms  said they would virtually halt exports of products hit by the  new taxes.  "We wouldn be able to do business," said a spokesma

In [111]:
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_list = []
    for token in doc:
        lemmatized_list.append(token.lemma_)
    
    return lemmatized_list


len(lemmatize_text(normalized_text))



880

In [12]:
def tokenize_text(text):
    doc = nlp(text)
    tokenized_list = [token.text for token in doc]
    return tokenized_list
print(tokenize_text(u"Apples and oranges are similar. Boots and hippos aren't."))

['Apples', 'and', 'oranges', 'are', 'similar', '.', 'Boots', 'and', 'hippos', 'are', "n't", '.']


In [13]:
all_stopwords = nlp.Defaults.stop_words
"," in all_stopwords


False

In [14]:
def stopword_removal(text):
    all_stopwords = nlp.Defaults.stop_words
    
    #all_stopwords.remove('not')

    text_tokens = lemmatize_text(text)
    tokens_without_sw = [word for word in text_tokens if not word in all_stopwords]

    print(tokens_without_sw)

stopword_removal("The, and, if are stopwords, computer is not"  )

[',', ',', 'stopword', ',', 'computer']


In [116]:
def normalize_docs(documents):

    normalized_documents = []
    for doc in documents:
        doc = expand_contractions(doc)
        doc = ' '.join(re.sub(' +', ' ',doc.replace('\'s','').replace('\'t','').replace('&lt;','').replace(">","")).split('\n')[1:]).strip()
        normalized_documents.append(doc)
    return normalized_documents

    
list_normalized_text = normalize_docs(text_list)

df['text'] = list_normalized_text






In [124]:
df.to_excel('Assets/preprocessed_text.xlsx',index=False)

In [126]:
pd.read_excel('Assets/preprocessed_text.xlsx')

,ids,categories,text
0,test/14826,['trade'],Mounting trade friction between the U.S. And ...
1,test/14828,['grain'],A survey of 19 provinces and seven cities sho...
2,test/14829,"['crude', 'nat-gas']",The Ministry of International Trade and Indus...
3,test/14832,"['corn', 'grain', 'rice', 'rubber', 'sugar', '...",Thailands trade deficit widened to 4.5 billio...
4,test/14833,"['palm-oil', 'veg-oil']",Indonesia expects crude palm oil (CPO) prices...
...,...,...,...
10783,training/999,"['interest', 'money-fx']",The Bank of England said it had revised its f...
10784,training/9992,['earn'],Qtly div 25 cts vs 25 cts prior Pay April 13 ...
10785,training/9993,['earn'],Qtly div 12 cts vs 12 cts prior Pay April 21 ...
10786,training/9994,['earn'],"Shr loss six cts vs loss 18 cts Net loss 89,4..."


In [120]:

# def remove_special_characters(text, remove_digits=False):
#     pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
#     text = re.sub(pattern, '', text)
#     return text

# remove_special_characters("Well this was fun! What do you think?", 
#                           remove_digits=True)